# Code final pour obtenir les recommendations pour un groupe donné. 

In [1]:
import pandas as pd
from selenium import webdriver
import numpy as np
import time
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import operator
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
"""
Récupération de tous les Dataframes finaux issus du scraping et leur concaténation dans un DataFrame final qui
sera utilisé dans le programme. 
"""


path="C:/Users/Utilisateur/Dropbox/Formation professionnelle IA/Projet Final/dataset final/"
df1=pd.read_csv(path+"final1.csv")
df2=pd.read_csv(path+"final2.csv")
df3,df4=pd.read_csv(path+"final3.csv"),pd.read_csv(path+"final4.csv")
df5,df6=pd.read_csv(path+"final5.csv"),pd.read_csv(path+"final6.csv")
df7,df8=pd.read_csv(path+"final7.csv"),pd.read_csv(path+"final8.csv")
df9,df10=pd.read_csv(path+"final9.csv"),pd.read_csv(path+"final10.csv")
df11,df12=pd.read_csv(path+"final11.csv"),pd.read_csv(path+"final12.csv")
df13,df14,df15=pd.read_csv(path+"final13.csv"),pd.read_csv(path+"final14.csv"),pd.read_csv(path+"final15.csv")
df16,df17,df18=pd.read_csv(path+"final16.csv"),pd.read_csv(path+"final17.csv"),pd.read_csv(path+"final18.csv")

df123=pd.concat([df1,df2,df3],ignore_index=True)
df456=pd.concat([df4,df5,df6],ignore_index=True)
df789=pd.concat([df7,df8,df9],ignore_index=True)
df101112=pd.concat([df10,df11,df12],ignore_index=True)
df131415=pd.concat([df13,df14,df15],ignore_index=True)
df161718=pd.concat([df16,df17,df18],ignore_index=True)


In [3]:
dfinal=pd.concat([df123,df456,df789,df101112,df131415,df161718],ignore_index=True)
dfinal.shape

(683, 8)

In [4]:
#Code pour le preprocessing des colonnes "lyrical themes" et "genre"

stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   
  
def clean_txt(text):
    clean_text = []
    clean_text2 = []
    text = re.sub("'", "",text)
    text=re.sub("(\\d|\\W)+"," ",text) 
    #text = text.replace("nbsp", "")
    clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
    clean_text2 = [word for word in clean_text if black_txt(word)]
    return " ".join(clean_text2)


In [5]:
#Transformation des valeurs nulles dans la colonne "lyrical themes"

dfinal['lyrical themes']=dfinal['lyrical themes'].fillna("unknown")

In [6]:
#Suppression des colonnes aux résultats incertains après copie du DataFrame final. 

erreur=list(dfinal[dfinal['favoris']>=100]['nom'])
index_delete=[]
df2=dfinal.copy()
for i in range(len(erreur)):
    elem=list(dfinal[dfinal["nom"]==erreur[i]].index)[0]
    df2.drop([elem],0,inplace=True)
df2=df2.reset_index(drop=True)

In [7]:


def meilleurs_reco_lyrics(elem,nombre_de_reco):
    lyrics2=df2.copy()
    lyrics2["lyrical themes"]=lyrics2["lyrical themes"].apply(clean_txt)
    vectorizer = CountVectorizer()
    X_lyrics= vectorizer.fit_transform((lyrics2['lyrical themes']))
    cosine_sim = cosine_similarity(X_lyrics)
    elem_index=lyrics2[lyrics2["nom"]==elem].index
    similar_movies = list(cosine_sim[elem_index][0])
    a=lyrics2["nom"]
    b=similar_movies
    sim={a[i]:b[j] for i in range(len(a)) for j in range(len(b)) if i==j}
    sortedDict = sorted(sim.items(), key=operator.itemgetter(1),reverse=True)
    sortedDict=[i for i in sortedDict if i[1]!=0]
    return sortedDict[1:nombre_de_reco+1]

In [8]:

def meilleurs_reco_genre(elem,nombre_de_reco):
    genre2=df2.copy()
    genre2["genre"]=genre2["genre"].apply(clean_txt)
    vectorizer = CountVectorizer()
    X_genre= vectorizer.fit_transform((genre2['genre']))
    cosine_sim = cosine_similarity(X_genre)
    elem_index=genre2[genre2["nom"]==elem].index
    similar_movies = list(cosine_sim[elem_index][0])
    a=genre2["nom"]
    b=similar_movies
    sim={a[i]:b[j] for i in range(len(a)) for j in range(len(b)) if i==j}
    sortedDict = sorted(sim.items(), key=operator.itemgetter(1),reverse=True)
    sortedDict=[i for i in sortedDict if i[1]!=0]
    return sortedDict[1:nombre_de_reco+1]

In [9]:
noms=list(df2["nom"])
elem=input("Veuillez entrer votre groupe avec des majuscules au début de chaque mot:")
if elem not in noms:
    print("Ce groupe n'est pas disponible. Veuillez réessayer!!!")
    elem=input("Veuillez entrer votre groupe:")
    
n=int(input("Combien de recommandations voulez-vous ?"))
b=list(df2[df2["nom"]==elem]["lyrical themes"])[0]
if b!="unknown":
    resultat3=meilleurs_reco_lyrics(elem,n)
    resultat4=meilleurs_reco_genre(elem,n)
    liste1=[resultat3[i][0] for i in range(len(resultat3))]
    liste2=[resultat4[i][0] for i in range(len(resultat4))]
    print("Voici vos recommandations basées sur les paroles de",elem,":",liste1)
    print("Voici vos recommandations basées sur le genre de",elem,":",liste2)
else:
    resultat4=meilleurs_reco_genre(elem,n)
    liste2=[resultat4[i][0] for i in range(len(resultat4))]
    print("Voici vos recommandations basées sur le genre de",elem,":",liste2)

Veuillez entrer votre groupe avec des majuscules au début de chaque mot:Kataklysm
Combien de recommandations voulez-vous ?10
Voici vos recommandations basées sur les paroles de Kataklysm : ['Arafel', 'Khors', 'Arghoslent', 'Grave Digger', 'Iced Earth', 'Tierra Santa', 'Abbath', 'ADX', 'Supuration', 'Oranssi Pazuzu']
Voici vos recommandations basées sur le genre de Kataklysm : ['Miseration', 'Illdisposed', 'Blood Stain Child', 'Anterior', 'Arghoslent', "Be'lakor", 'Dark Tranquillity', 'Tracedawn', 'Mors Principium Est', 'Aborted']
